<a href="https://colab.research.google.com/github/bxck75/Cartoonizer/blob/master/CartoonizerV6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cartoon style on image folder



## Setup

In [0]:
import os
os.chdir('/content')
storage_mode="colab" #@param ['colab','drive']
# Handle storagemode choice
if storage_mode == 'colab':
  root = '/content/Cartoonizer/' 
else:
  # sync google drive
  from google.colab import drive
  root = '/content/drive/My Drive/' # google drive root
  # if drive is needed uncomment
  drive.mount('/content/drive',force_remount=True)
# check if folder exists
if not os.path.exists(root):
   os.mkdir(root,777)
# go to root    
os.chdir(root)
!rm -r ../sample_data


## Imports!

In [0]:
# set video details
# video_name='lucky_chops' 
# vid='https://www.youtube.com/watch?v=gFrCyU_1zw0' # orbital - the box
# video_name='busker' 
# vid="https://www.youtube.com/watch?v=l1wN6HuWrSE" # oizo 
# video_name='techno_dark_'
# vid='https://www.youtube.com/watch?v=lQnzf5vrK5U'
# video_name='techno_dark_2'
# vid='https://www.youtube.com/watch?v=aAMduklK3iU'
# video_name='piano' 
# vid="https://www.youtube.com/watch?v=n-BtBRS1OoA" # righthere
video_name='loopstation' 
vid="https://www.youtube.com/watch?v=x1m4-iZ-7CM"
# video_name="beatbox" 
# vid="https://www.youtube.com/watch?v=DFjIi2hxxf0" # our love 
# video_name='harmbeatbox' 
# vid="https://www.youtube.com/watch?v=Jbgb6Mz95rc"
# video_name='trainharmonica'
# vid="https://www.youtube.com/watch?v=0dRwYEm3F2o"
# video_name='beatbox1'
# vid="https://www.youtube.com/watch?v=oBjGRZgHgIA"
# video_name='harmonics'
# vid="https://www.youtube.com/watch?v=K-PhBryFuIM"
# set video details
# video_name='lucky_chops' 
# vid='https://www.youtube.com/watch?v=gFrCyU_1zw0' # orbital - the box
# video_name='busker' 
# vid="https://www.youtube.com/watch?v=l1wN6HuWrSE" # oizo 
# video_name='techno_dark_'
# vid='https://www.youtube.com/watch?v=lQnzf5vrK5U'
# video_name='techno_dark_2'
# vid='https://www.youtube.com/watch?v=aAMduklK3iU'
# video_name='piano' 
# vid="https://www.youtube.com/watch?v=n-BtBRS1OoA" # righthere
# video_name='this_land' 
# vid="https://www.youtube.com/watch?v=FZN-yJv82Rs"
# video_name="our_love" 
# vid="https://www.youtube.com/watch?v=eyGWBTCZ3mk" # our love 
# video_name='harmbeatbox' 
# vid="https://www.youtube.com/watch?v=Jbgb6Mz95rc"
# video_name='trainharmonica'
# vid="https://www.youtube.com/watch?v=0dRwYEm3F2o"
# video_name='beatbox1'
# vid="https://www.youtube.com/watch?v=oBjGRZgHgIA"
# video_name='harmonics'
# vid="https://www.youtube.com/watch?v=K-PhBryFuIM"

# prep the folders for the video frames
if not os.path.exists(root+video_name):
  os.mkdir(root+video_name,777)
if not os.path.exists(root+"processed_"+video_name):
  os.mkdir(root+"processed_"+video_name,777)
# frames folders
if not os.path.exists(root+video_name+'/_frames'):
  os.mkdir(root+video_name+'/_frames',777)
if not os.path.exists(root+"processed_"+video_name +"/_frames"):
  os.mkdir(root+"processed_"+video_name +"/_frames",777)
# set the folder name in vars for later use
video_root = root+video_name
frames_dir =root+video_name+'/_frames'
proc_frames_dir = root+"processed_"+video_name+'/_frames'

# imports
import time,os,glob,cv2
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import clear_output, display, Image
from moviepy.editor import VideoFileClip
# pip stuff
!pip install youtube-dl imageio
!pip install tensorflow-gpu==2.0.0-alpha0
!pip install ffmpeg
import imageio  
import tensorflow as tf
tf.__version__

# Gitgo class
class GitGo():
  def __init__(self,rep,chdir=True,path='/content/'):
    self.rep = rep
    self.chdir = chdir
    self.path = path
    if self.rep == 'help':
      self.help()
    self.go() 
      
  def help(self):
    print("* pulls git rep and shows files \
            * returns root path for the repository \
            * Function needs repository <user>/<repository name> combination\
            * Switch chdir and define the rootpath for the repository\
            * Use : GitGo('<rep owner>/<rep name>', <True/False>, <root path>)\
          ")
    sys.exit()
    
  def go(self):
    self.rep=self.rep.split('/')
    # change folder check
    if self.chdir ==True:
      #Switch to path
      os.chdir(self.path)
    # pull get the git rep
    os.system('git clone https://github.com/'+self.rep[0]+'/'+self.rep[1]+'.git')
    # Set the return value for rep rootpath
    self.PATH=self.path+self.rep[1]
    # show imported files
    os.system('ls ' + self.PATH)
    
  def __repr__(self):
    return self.PATH
  
# apt suff
!apt update
!apt install tree

# git reps
os.chdir(str(GitGo('mnicnc404/CartoonGan-tensorflow')))
os.chdir(str(GitGo('keras-team/keras-contrib')))
#convert keras to tf
os.system('python convert_to_tf_keras.py && USE_TF_KERAS=1; python setup.py install')

#function defines
def resize(image,window_height = 800):
    aspect_ratio = float(image.shape[1])/float(image.shape[0])
    window_width = window_height/aspect_ratio
    image = cv2.resize(image, (int(window_height),int(window_width)))
    return image


clear_output()

In [3]:
!ls

build		 convert_to_tf_keras.py  keras_contrib		   pytest.ini
CODEOWNERS	 dist			 keras_contrib.egg-info    README.md
contrib_docs	 examples		 LICENSE		   setup.py
CONTRIBUTING.md  GUIDELINES.md		 PULL_REQUEST_TEMPLATE.md  tests


## Rip the video and audio

In [0]:
# frame rate
framerate = 0.1
# set max img height
max_size= 600 # frame height limit before it wil be resized
print('sampling rate for frames = '+str(round(1/framerate,2)) + ' per second.')

# Set i/o folders

img_in_folder = frames_dir # video_root +"_frames"
img_out_folder = proc_frames_dir # root+"processed_"+video_name +"_frames"

# set original filenames
video_file= root+ '/' +video_name +'/'+ 'video.mp4'
audio_file= root+ '/' +video_name +'/'+ 'audio.mp3'

# rip video from youtube
os.system('echo youtube-dl '+vid+' --list-formats')

# get the mp4
print('Ripping video : '+ vid )
!youtube-dl {vid} --output '{video_file}' --format mp4

# split of the mp3
clip = VideoFileClip(video_file)
print('Ripping audio...' + audio_file)
clip.audio.write_audiofile(audio_file)

#capture the video frames
vidcap = cv2.VideoCapture(video_file)

# count total frames
vidc = imageio.get_reader(video_file,  'ffmpeg')
num_frames=vidc._meta['nframes']

# # make the frames folder
# if not os.path.exists(frames_dir):
#   os.mkdir(root+video_name+'_frames')
  
# frame grab function
def getFrame(sec):
  vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
  hasFrames,image = vidcap.read()
  if hasFrames:
    cv2.imwrite(frames_dir+"image"+str(count)+".jpg", image)     # save frame as JPG file
  return hasFrames

sec = 0
FrameRate = framerate
count=1
success = getFrame(sec)
# As long as there are frames
print("Splitting in frames...this might take a while...")
while success:
  count = count + 1
  frames_left = num_frames -1
  # print('frames to go ' + str(frames_left))
  sec = sec + FrameRate
  sec = round(sec, 2)
  success = getFrame(sec)
  
clear_output()

## Set the style 

In [0]:
# Pick a style
styles = "hayao" #@param ["shinkai", "hayao", "hosoda", "paprika"]

## Set Dimensions

In [6]:
# generate the movie sequence
searchedfiles = glob.glob(img_in_folder+'/*.jpg')
files = sorted( searchedfiles, key = lambda file: os.path.getctime(file))

files_sorted=[]
combined_sorted=[]
# fill the frame list before the processed image even beeing there
for i in files:
  # split off the filename
  isa = i.split('/')
  # append it with the processed folder path and file definition
  combined_sorted.append("file '"+video_root+"/comparison/"+isa[len(isa)-1]+"'\n")
  files_sorted.append("file '"+proc_frames_dir+"/"+styles+"/"+isa[len(isa)-1]+"'\n")
print(combined_sorted)
print(files_sorted)

# for file in files check height
for f in files:
  img = cv2.imread(f)
  h,w,c = img.shape
  # Resize if needed
  if h > max_size:
    img_resized = resize(img,window_height = max_size)
    rh,rw,rc = img_resized.shape
    cv2.imwrite(f,img_resized)

[]
[]


## Run the Cartoonizer

In [7]:
# run the Cartoonizer
!python ../CartoonGan-tensorflow/cartoonize.py \
  --comparison_view horizontal \
  --keep_original_size \
  --batch_size 8 \
  --styles '{styles}' \
  --input_dir '{img_in_folder}' \
  --output_dir '{img_out_folder}'


[2019-08-30 20:45:36] [Cartoonizer] [INFO] Transformed images will be saved to `/content/Cartoonizer/processed_loopstation/_frames` folder.
[2019-08-30 20:45:37] [Cartoonizer] [INFO] Cartoonizing images using hayao style...
[2019-08-30 20:45:37] [Cartoonizer] [INFO] Preparing to transform 0 images from `/content/Cartoonizer/loopstation/_frames` directory...
Transforming: 0it [00:00, ?it/s]
[2019-08-30 20:45:37] [Cartoonizer] [INFO] Total processing time: 0:00:01.556435


## Forming the new video 

In [0]:
with open(root+'/paths.txt', 'w') as in_files:
  in_files.writelines(fn for fn in files_sorted)
with open(root+'/compare_paths.txt', 'w') as in_files:
  in_files.writelines(fn for fn in combined_sorted)
  
!ffmpeg -y -r 1/{framerate} -f concat -safe 0 -i  '{video_root}'"paths.txt" -c:v libx264 -vf "fps=1/"{framerate}",format=yuv420p" '{video_root}'"processed_"{video_name}".mp4"
!ffmpeg -i '{root}'"processed_"'{video_name}'".mp4" -i '{audio_file}' -shortest '{video_root}'"final_processed_"'{video_name}'"_vid.mp4"

!ffmpeg -y -r 1/{framerate} -f concat -safe 0 -i  '{cideo_root}'"compare_paths.txt" -c:v libx264 -vf "fps=1/"{framerate}",format=yuv420p" '{video_root}'"processed_compare_"{video_name}".mp4"
!ffmpeg -i '{root}'"processed_compare_"'{video_name}'".mp4" -i '{audio_file}' -shortest '{video_root}'"final_processed_compare_"'{video_name}'"_vid.mp4"

In [0]:
# zip the complete project for download

!zip -r /content/Cartoonizer_sources.zip /content

## View results

In [0]:
# Debug displays all frames.......LAGS LIKE a MOFO!
# folders = [f for f in glob.glob(img_out_folder + "**/*", recursive=True)]
# for d in folders:
#   !tree '{d}'
#   files = [f for f in glob.glob( d + "**/*.jpg", recursive=True )]
#   for f in files:
#       display(Image(f))


In [0]:
# # experimental filters and effects after this cel!
# import moviepy
# from moviepy.editor import VideoFileClip, clips_array, vfx
# print(dir( moviepy.editor))
# # def invert_green_blue(image):
# #     return image[:,:,[0,2,1]]
# # def invert_red_green(image):
# #     return image[:,[0,2,1],:

# clip1 = VideoFileClip('/content/Cartoonizer/final_processed_compare_'+ video_name + "_vid.mp4").margin(1) # add 10px contour
# clip1 = clip1.fx( vfx.colorx, 1.3)
# # clip2 = clip1.fx( vfx.mirror_y)
# # clip3 = clip1.fx( vfx.mirror_x)
# # clip4 = clip1.fx( vfx.mirror_y)
# # clip5 = clip1.fx( vfx.colorx, 1.3) # up 40%
# # modifiedClip1 = clip1.fl_image( invert_red_green )
# # modifiedClip2 = clip1.fl_image( invert_green_blue )
# final_clip = clips_array([[clip1]])
# final_clip.write_videofile("/content/Cartoonizer/fx_stack"+video_name +".mp4")




In [0]:

from google.colab import drive
import os
# video_name='trainrolsin'
drive.mount('/content/drive')
# if not os.path.exists("/content/drive/My Drive/"+video_name):
#   os.mkdir("/content/drive/My Drive/"+video_name,777)
#   os.mkdir("/content/drive/My Drive/"+video_name+"_frames",777)
 
# !mv '/content/Cartoonizer/processed_'{video_name}'_frames/*' '/content/drive/My Drive/'{video_name}'_frames/'

In [0]:
# %from moviepy.editor import *
# clip = (VideoFileClip( root+ video_name " + ".mp4")
# #         .fx( vfx.resize, width=460) # resize (keep aspect ratio)
# #         .fx( vfx.speedx, 2) # double the speed
#         .fx( vfx.colorx, 1.3)) # darken the picture

# clip.write_videofile("brighness_video_name.mp4")

In [0]:
# from moviepy.editor import VideoFileClip, concatenate_videoclips
# clip1 = VideoFileClip( root + video_name " + ".mp4")
# clip2 = VideoFileClip( root + video_name " + ".mp4").subclip(50,60)
# clip3 = VideoFileClip( root + video_name " + ".mp4")
# # final_clip = concatenate_videoclips([clip1,clip2,clip3])
# final_clip = clips_array([[clip1, clip2],
#                           [clip3, clip4]])
# final_clip.write_videofile(video_name+"_concatenation.mp4")

In [0]:
# All input widgets!
sys.exit()
#@title Example form fields
#@markdown Forms support many types of fields.

no_type_checking = 'name'  #@param
string_type = 'example'  #@param {type: "string"}
slider_value = 142  #@param {type: "slider", min: 100, max: 200}
number = 102  #@param {type: "number"}
date = '2010-11-05'  #@param {type: "date"}
pick_me = "monday"  #@param ['monday', 'tuesday', 'wednesday', 'thursday']
select_or_input = "apples" #@param ["apples", "bananas", "oranges"] {allow-input: true}
#@markdown ---


In [0]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [0]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))